In [1]:
import pandas as pd
import numpy as np
from math import pi
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import font_manager, rc
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import norm

## 클러스터링 코드도 필요해서 넣었슴

In [3]:
# 데이터셋을 읽어옵니다.
df = pd.read_csv('피벗_ver4.csv')

# '지역' 열을 제외한 나머지 데이터를 추출합니다.
df_data = df.pivot(index='지역', columns='피벗 필드명', values='피벗 필드값')

# 데이터 정규화를 위해 StandardScaler를 적용합니다.
# scaler = StandardScaler()
# scaled_data = scaler.fit_transform(df_data)

# K-Means 클러스터링을 적용합니다.
num_clusters = 4  # 클러스터 개수를 지정
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
labels = kmeans.fit_predict(df_data)

# 클러스터링 결과 딕셔너리에 저장
cluster_data = {}
for cluster_idx in range(kmeans.n_clusters):
    cluster_data[cluster_idx] = []
for idx, cluster_idx in enumerate(labels):
    cluster_data[cluster_idx].append(df_data.index[idx])

# 클러스터링 결과를 데이터프레임에 추가합니다.
df_data['Cluster'] = labels

# 클러스터별로 지역들을 그룹화하여 레이더 차트를 그립니다.
clusters = df_data['Cluster'].unique()
for cluster in clusters:
    cluster_df = df_data[df_data['Cluster'] == cluster].drop(columns='Cluster')
    
    # 레이더 차트를 그립니다.
'''    num_features = len(cluster_df.columns)
    angles = [n / float(num_features) * 2 * pi for n in range(num_features)]
    angles += angles[:1]  # 첫번째 값을 마지막으로 이동하여 레이더 차트를 닫아줍니다.
    rc('font', family='AppleGothic')

    plt.figure(figsize=(8, 8))
    for index, row in cluster_df.iterrows():
        values = row.tolist()
        values += values[:1]  # 첫번째 값을 마지막으로 이동하여 레이더 차트를 닫아줍니다.
        plt.polar(angles, values, marker='o', label=index)

    plt.fill(angles, values, alpha=0.25)
    plt.title(f'Radar Chart Clustering (Cluster {cluster})')
    plt.xticks(angles[:-1], cluster_df.columns)
    plt.yticks(np.arange(0, 101, 20))
    plt.ylim(0, 4) #plt.ylim(0, 1)
    plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1))
    plt.show()'''

print(cluster_data)
df_cluster = pd.DataFrame([(data, cluster_data) for cluster_data, regions in cluster_data.items() for data in regions],
                  columns=['Data', 'Cluster'])

#광역시의 구 제외
"""remove = ['강북구', '광산구', '구로구', '금정구', '동대문구', '동래구', '사하구', '양천구', '은평구', '광진구', '서대문구']
for rem in remove:
    df_cluster = df_cluster.drop(df_cluster[df_cluster['Data'] == rem].index)
df_cluster.reset_index(drop=True, inplace=True)
"""
df_cluster

{0: ['강진군', '고흥군', '괴산군', '구례군', '금산군', '남원시', '보성군', '산청군', '신안군', '영광군', '영덕군', '영천시', '예천군', '완도군', '울진군', '음성군', '인제군', '장성군', '장수군', '진도군', '진안군', '철원군', '청송군', '청양군', '칠곡군', '함양군', '해남군'], 1: ['강북구', '광산구', '구로구', '군포시', '금정구', '김천시', '동대문구', '동래구', '무주군', '사천시', '사하구', '아산시', '양천구', '오산시', '은평구', '의정부시'], 2: ['거제시', '관악구', '광주시', '광진구', '서대문구', '양평군', '춘천시', '통영시', '파주시'], 3: ['고령군', '곡성군', '나주시', '논산시', '밀양시', '봉화군', '성주군', '예산군', '완주군', '의성군', '익산시', '정읍시', '진주시', '하동군', '함안군', '합천군', '화천군']}


,Data,Cluster
0,강진군,0
1,고흥군,0
2,괴산군,0
3,구례군,0
4,금산군,0
...,...,...
64,진주시,3
65,하동군,3
66,함안군,3
67,합천군,3


## 워케이션 적합 지역 선정

In [6]:
#csv 불러오기 및 결측지역 제거
pls = pd.read_csv('PLS.csv', index_col=0)
pls = pls.drop(pls[pls['지역'] == '31**'].index)
pls = pls.drop(pls[pls['지역'] == '***'].index)

#같은 지역코드를 가진 것끼리 groupby
pls2 = pls.groupby('지역').sum().reset_index()
pls2 = pls2.drop(['기간','교통','관광쇼핑','패션쇼핑','종교활동','종합쇼핑','체험','미술공예참여','악기연주','인터넷게임','스포츠용품구매','여행사','독서','운동경기관람','음악감상','스키'], axis=1)
#활동별 이용건수 z-score 정규화
pls2[pls2.columns.difference(['지역'])] = pls2[pls2.columns.difference(['지역'])].apply(lambda x: (x - x.mean()) / x.std())

#적합도 계산 -> 0~1값으로 스케일링
pls2['적합도'] = (
    (pls2['골프']*0.807 + pls2['레저스포츠']*0.906 + pls2['자전거']*0.807 + pls2['헬스']*0.888) * -0.165 +
    (pls2['공연관람']*0.835 + pls2['사진촬영']*0.901) * 0.06 +
    (pls2['숙박']) * 0.034 +
    (pls2['교육훈련']*0.898 + pls2['목욕']*0.891 + pls2['미용']*0.96 + pls2['애완동물돌보기']*0.945 + pls2['외식']*0.827 + pls2['유흥']*0.885) * -0.09
)
pls2['적적합도'] = norm.cdf(pls2['적합도'])

#pls2파일 저장해주고 지역코드를 지역명으로 변경 후 다시불러오기
fit = pd.read_csv('적합성.csv', index_col=0)

#기존의 지역별 클러스터 데이터프레임과 합쳐주기
for i in range(69):
    region = df_cluster.loc[i].Data #강진군
    clu = df_cluster.loc[i].Cluster #0
    row_name = fit.loc[fit['지역명'] == region].index[0]
    df_cluster.at[i, '적합도'] = fit.loc[row_name, '적적합도']

pd.set_option('display.max_rows', 69)
df_cluster.to_csv('cluster&fitness.csv', encoding='cp949')
df_cluster.sort_values(by='적합도')

,Data,Cluster,적합도
18,장수군,0,0.231938
57,봉화군,3,0.232825
20,진안군,0,0.233102
23,청양군,0,0.234067
53,곡성군,3,0.234478
8,신안군,0,0.234901
7,산청군,0,0.235201
6,보성군,0,0.235362
52,고령군,3,0.235762
3,구례군,0,0.235849


## 기대효과 분석 시작

In [8]:
df_spend = pd.read_csv('소비액+인구수+나눔.csv')
df_spend.drop('코드', axis=1, inplace=True)
df_spend['시군구명'] = df_spend['시군구명'].str.replace(" ", "")
df_spend

,시도명,시군구명,소비액,인구,나눔
0,서울,종로구,471861793,140397,3360.91
1,서울,중구,1563239458,121441,12872.42
2,서울,용산구,379294352,217288,1745.58
3,서울,성동구,583839393,279374,2089.81
4,서울,광진구,86920699,336169,258.56
...,...,...,...,...,...
224,경남,함양군,9083607,37294,243.57
225,경남,거창군,58396255,60356,967.53
226,경남,합천군,17663293,41741,423.16
227,제주,제주시,6787607671,492546,13780.66


In [9]:
#변수 정의
effect_loss = 52103 #kosis 1인당 민간소비지출액 / 360일
effect_gain = 130804 #국민여행조사보고 당일치기 소비액, 안당일치기 소비액 가중치 평균
worker = 20000000

## 1. 워케이션 이동 인구: if) 각 지역의 0.1%가 워케이션을 떠난다면?

In [10]:
#지역 사람들이 워케이션을 떠남으로써 발생하는 지역 손실
df_spend2 = df_spend.copy()
df_spend2['소비액'] = df_spend2['소비액'] - df_spend2['인구']*0.001*effect_loss
#df_spend2['소비액'] = df_spend2['소비액'].astype(int)
df_spend2

,시도명,시군구명,소비액,인구,나눔
0,서울,종로구,4.645467e+08,140397,3360.91
1,서울,중구,1.556912e+09,121441,12872.42
2,서울,용산구,3.679730e+08,217288,1745.58
3,서울,성동구,5.692832e+08,279374,2089.81
4,서울,광진구,6.940529e+07,336169,258.56
...,...,...,...,...,...
224,경남,함양군,7.140478e+06,37294,243.57
225,경남,거창군,5.525153e+07,60356,967.53
226,경남,합천군,1.548846e+07,41741,423.16
227,제주,제주시,6.761945e+09,492546,13780.66


## 2. 워케이션 이동 양상: if) 클러스터에도 고르게, 클러스터 안에서도 고르게 이동한다면?

In [26]:
#클러스터당 이동 인구 계산
df_spend3 = df_spend2.copy()
##woca_people = (df_spend3['인구'].sum()*0.001 / num_clusters).astype(int)
woca_people = 10440000  ##################################################영향 받은 방문자 수
w_nature = woca_people * 8/21 #0
w_food = woca_people * 6/21 #2
w_sports = woca_people * 6/21 #1
w_history = woca_people * 1/21 #3

#기존 df와 지역별 클러스터 df 합쳐주기
for i in range(len(df_cluster)):
    region = df_cluster.loc[i].Data
    clu = df_cluster.loc[i].Cluster
    row_name = df_spend3.loc[df_spend3['시군구명'] == region].index[0]
    df_spend3.at[row_name, '클러스터'] = clu
    df_spend3 = df_spend3.fillna(-1)
    #df_spend3['클러스터'] = df_spend3['클러스터'].astype(int)
df_spend3

#클러스터별 이동인구와 이득
move = pd.DataFrame(df_spend3['클러스터'].value_counts())
#move['이동인구'] = ((woca_people/4) / move['클러스터']).astype(int)
move.at[-1,'이동인구'] = 0
move.at[0, '이동인구'] = w_nature / move.loc[0, '클러스터']
move.at[1, '이동인구'] = w_sports / move.loc[1, '클러스터']
move.at[2, '이동인구'] = w_food / move.loc[2, '클러스터']
move.at[3, '이동인구'] = w_history / move.loc[3, '클러스터']
move['이득'] = move['이동인구'] * effect_gain
move

#이동 후 지수변화 계산
for i in range(len(df_spend3)):
    cl = df_spend3.loc[i, '클러스터']
    df_spend3.at[i, '소비액'] += move.loc[cl, '이득']
df_spend3['나눔(이동후)'] = (df_spend3['소비액'] / df_spend3['인구']).round(2)
df_spend3['지수변화'] = df_spend3['나눔(이동후)'] - df_spend3['나눔']
df_spend3

,시도명,시군구명,소비액,인구,나눔,클러스터,나눔(이동후),지수변화
0,서울,종로구,4.645467e+08,140397,3360.91,-1.0,3308.81,-52.10
1,서울,중구,1.556912e+09,121441,12872.42,-1.0,12820.32,-52.10
2,서울,용산구,3.679730e+08,217288,1745.58,-1.0,1693.48,-52.10
3,서울,성동구,5.692832e+08,279374,2089.81,-1.0,2037.71,-52.10
4,서울,광진구,4.342159e+10,336169,258.56,2.0,129165.95,128907.39
5,서울,동대문구,2.441171e+10,338115,129.30,1.0,72199.42,72070.12
6,서울,중랑구,9.263536e+08,384272,2462.77,-1.0,2410.67,-52.10
7,서울,성북구,2.689606e+08,429142,678.84,-1.0,626.74,-52.10
8,서울,강북구,2.446479e+10,291384,323.85,1.0,83960.64,83636.79
9,서울,도봉구,2.158217e+09,308986,7036.94,-1.0,6984.84,-52.10


## --------------------버그방지선--------------------

In [27]:
move

,클러스터,이동인구,이득
-1.0,160,0.000000,0.000000e+00
0.0,27,147301.587302,1.926764e+10
3.0,17,29243.697479,3.825193e+09
1.0,16,186428.571429,2.438560e+10
2.0,9,331428.571429,4.335218e+10


In [28]:
before = pd.read_csv('before.csv', encoding='cp949')
pd.set_option('display.max_rows', 250)
before

,기초지자체,기타관광,레저스포츠,문화관광,쇼핑,숙박,역사관광,음식,자연관광,체험관광,외부방문자 수
0,서울특별시 강남구,4681899.0,258510.0,27999477.0,29437557.0,28349901.0,2173397.0,98580294.0,0.0,7660.0,191488692.0
1,서울특별시 강동구,86946.0,262613.0,5863096.0,9500638.0,2338230.0,877446.0,25413995.0,0.0,17301.0,44360264.0
2,서울특별시 강북구,419085.0,57251.0,2505485.0,6420236.0,3277662.0,3538251.0,11829140.0,154548.0,564.0,28202221.0
3,서울특별시 강서구,25319180.0,836708.0,5323442.0,16538216.0,4454421.0,299757.0,15966889.0,0.0,12375.0,68751676.0
4,서울특별시 관악구,20944.0,116978.0,4175451.0,16565924.0,4156550.0,698803.0,24840712.0,502648.0,3576.0,51082096.0
5,서울특별시 광진구,3321744.0,356093.0,16988076.0,9025195.0,11145648.0,578576.0,17957341.0,23859.0,250518.0,59647049.0
6,서울특별시 구로구,10567.0,32230.0,6617155.0,25993097.0,6166465.0,123636.0,12711235.0,1180880.0,1057.0,52835793.0
7,서울특별시 금천구,0.0,10296.0,2098084.0,15233085.0,2391662.0,416871.0,6242769.0,1056.0,6864.0,26400951.0
8,서울특별시 노원구,481727.0,2617319.0,4513052.0,28106131.0,960941.0,381661.0,13174592.0,42742.0,6537.0,50284702.0
9,서울특별시 도봉구,58644.0,107078.0,2411479.0,11375135.0,1254043.0,297148.0,10595482.0,81159.0,524.0,26180431.0


In [29]:
visit = df_cluster.copy()

for i in range(len(visit)):
    if visit.loc[i, 'Cluster'] == 0:
        visit.at[i, 'visitor'] = move.loc[0, '이동인구']
    elif visit.loc[i, 'Cluster'] == 1:
        visit.at[i, 'visitor'] = move.loc[1, '이동인구']
    elif visit.loc[i, 'Cluster'] == 2:
        visit.at[i, 'visitor'] = move.loc[2, '이동인구']
    else:
        visit.at[i, 'visitor'] = move.loc[3, '이동인구']
visit

,Data,Cluster,적합도,visitor
0,강진군,0,0.239388,147301.587302
1,고흥군,0,0.248022,147301.587302
2,괴산군,0,0.240954,147301.587302
3,구례군,0,0.235849,147301.587302
4,금산군,0,0.243573,147301.587302
5,남원시,0,0.261939,147301.587302
6,보성군,0,0.235362,147301.587302
7,산청군,0,0.235201,147301.587302
8,신안군,0,0.234901,147301.587302
9,영광군,0,0.245984,147301.587302


In [31]:
ww_nature = w_nature/192
ww_sports = w_sports/192
ww_food = w_food/192
ww_history = w_history/192


    
before['자연관광'] -= ww_nature
before['레저스포츠'] -= ww_sports
before['음식'] -= ww_food
before['역사관광'] -= ww_history

for i in range(len(visit)):
    for j in range(len(before)):
        if visit.loc[i, 'Data'] in before.loc[j, '기초지자체']:
            if visit.loc[i, 'Cluster'] == 0:
                before.at[j, '자연관광'] += visit.loc[i, 'visitor'].round()
            elif visit.loc[i, 'Cluster'] == 1:
                before.at[j, '레저스포츠'] += visit.loc[i, 'visitor'].round()
            elif visit.loc[i, 'Cluster'] == 2:
                before.at[j, '음식'] += visit.loc[i, 'visitor'].round()
            elif visit.loc[i, 'Cluster'] == 3:
                before.at[j, '역사관광'] += visit.loc[i, 'visitor'].round()
            before.at[j, '자연관광'] += ww_nature
            before.at[j, '레저스포츠'] += ww_sports
            before.at[j, '음식'] += ww_food
            before.at[j, '역사관광'] += ww_history
            
before.loc[before['자연관광'] < 0, '자연관광'] = 0
before.loc[before['레저스포츠'] < 0, '레저스포츠'] = 0
before.loc[before['음식'] < 0, '음식'] = 0
before.loc[before['역사관광'] < 0, '역사관광'] = 0

before.to_csv('after1044.csv', encoding='cp949')
before

,기초지자체,기타관광,레저스포츠,문화관광,쇼핑,숙박,역사관광,음식,자연관광,체험관광,외부방문자 수
0,서울특별시 강남구,4681899.0,2.274386e+05,27999477.0,29437557.0,28349901.0,2.168218e+06,9.854922e+07,0.000000e+00,7660.0,191488692.0
1,서울특별시 강동구,86946.0,2.315416e+05,5863096.0,9500638.0,2338230.0,8.722674e+05,2.538292e+07,0.000000e+00,17301.0,44360264.0
2,서울특별시 강북구,419085.0,4.301090e+05,2505485.0,6420236.0,3277662.0,3.538251e+06,1.182914e+07,1.545480e+05,564.0,28202221.0
3,서울특별시 강서구,25319180.0,8.056366e+05,5323442.0,16538216.0,4454421.0,2.945784e+05,1.593582e+07,0.000000e+00,12375.0,68751676.0
4,서울특별시 관악구,20944.0,1.169780e+05,4175451.0,16565924.0,4156550.0,6.988030e+05,2.550357e+07,5.026480e+05,3576.0,51082096.0
5,서울특별시 광진구,3321744.0,3.560930e+05,16988076.0,9025195.0,11145648.0,5.785760e+05,1.862020e+07,2.385900e+04,250518.0,59647049.0
6,서울특별시 구로구,10567.0,4.050880e+05,6617155.0,25993097.0,6166465.0,1.236360e+05,1.271124e+07,1.180880e+06,1057.0,52835793.0
7,서울특별시 금천구,0.0,0.000000e+00,2098084.0,15233085.0,2391662.0,4.116924e+05,6.211698e+06,0.000000e+00,6864.0,26400951.0
8,서울특별시 노원구,481727.0,2.586248e+06,4513052.0,28106131.0,960941.0,3.764824e+05,1.314352e+07,1.313429e+03,6537.0,50284702.0
9,서울특별시 도봉구,58644.0,7.600657e+04,2411479.0,11375135.0,1254043.0,2.919694e+05,1.056441e+07,3.973043e+04,524.0,26180431.0


## -------------------------------------------------------------------